In [ ]:
import time
import numpy as np
import pandas as pd
import missingno as msno
import matplotlib.pyplot as plt
import statsmodels.formula.api as sm
import seaborn as sns; sns.set()
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE
from sklearn.linear_model import LassoCV, LassoLarsCV, LassoLarsIC
from sklearn import datasets


In [ ]:
df = pd.read_csv("C:/Users/Hangyu/Desktop/loan.csv")     #change to your own path

In [ ]:
%matplotlib inline
msno.matrix(df)  
#The msno.matrix nullity matrix is a data-dense display which lets you quickly visually pick out patterns in data completion.
#visualize the missing data

In [ ]:
df.shape   #dimension of our dataset

In [ ]:
df.info()

In [ ]:
df.loan_status.value_counts()   #what we want to predict

In [ ]:
#Part 1, pre-processing

In [ ]:
#1-1, 
#Since the data can be devided into 2 groups, with co-borrowers or without co-borrowers
#application_type， Indicates whether the loan is an individual application or a joint application with two co-borrowers
df.application_type.value_counts()   
#only 511 has co-borrowers, so we drop all co-borrowers' information, and keep application_type(which can be a indicator for loan status)

In [ ]:
new_df = df.drop(['annual_inc_joint','dti_joint','verification_status_joint','acc_now_delinq','tot_coll_amt','tot_cur_bal','open_acc_6m','open_il_6m','open_il_12m',
                  'open_il_24m','mths_since_rcnt_il','total_bal_il','il_util','open_rv_12m','open_rv_24m','max_bal_bc','all_util',
                  'total_rev_hi_lim','inq_fi','total_cu_tl','inq_last_12m'], axis=1)
new_df.head()

In [ ]:
%matplotlib inline
msno.matrix(new_df)  

In [ ]:
#1-2. Check Missing Value
missing_portion = new_df.isnull().sum()/len(new_df)    #calculate the portion of missing value
print(missing_portion.sort_values(ascending=False))
#Here,pick a value and drop the factors base on the missing portion
#My initial thought was drop everything above the "emp_title" , but when I looked into the columns' description
#Factors like mths_since_last_record is definitely related to loan status, so I have to look into these factors 1 by 1

In [ ]:
#a. desc: Loan description provided by the borrower
#can be dropped, texture data, and lots of missing value
new_df_2 = new_df.drop(['desc'], axis=1)

In [ ]:
#b. mths_since_last_record: The number of months since the last public record.
#c. mths_since_last_major_derog: Months since most recent 90-day or worse rating
#d. mths_since_last_delinq: The number of months since the borrower's last delinquency.    
#can not be dropped, I guess the missing values should be replaced with "0" ***need further verification***
values = {'mths_since_last_record': 0, 'mths_since_last_major_derog': 0, 'mths_since_last_delinq': 0}

In [ ]:
new_df_3 = new_df_2.fillna(value=values)

In [ ]:
#e. next_pymnt_d: Next scheduled payment date
new_df_4 = new_df_3.drop(['next_pymnt_d'], axis=1)

In [ ]:
new_df_4.head()

In [ ]:
#1-3. Missing Value Continued.(replacing missing value)
df_no_missing = new_df_4.dropna()    #
%matplotlib inline
msno.matrix(df_no_missing)   #looks perfect, no missing value

In [ ]:
#2-1. drop texture data, and unnecessary data
#id: A unique LC assigned ID for the loan listing.
#member_id: A unique LC assigned Id for the borrower member.
#emp_title: The job title supplied by the Borrower when applying for the loan.*
#url: URL for the LC page with listing data.
#policy_code: "publicly available policy_code=1, new products not publicly available policy_code=2"
df_no_missing_1 = df_no_missing.drop(['id','member_id','emp_title','url','policy_code'], axis=1)

In [ ]:
#3-1. Status
df_status = df_no_missing_1
status_replace = {"loan_status" : { 'Current':1,'Issued':1,'Fully Paid':1,'In Grace Period':0,
                                   'Does not meet the credit policy. Status:Fully Paid':1,
                                   'Does not meet the credit policy. Status:Charged Off':0,
                                   'Default':0,
                                   'Late (31-120 days)':0,'Late (16-30 days)':0,'Charged Off':0 }}
                  #The Current is a great portion of the data, maybe we can do like this. "Good debt" vs "Bad debt"
df_status =df_status.replace(status_replace)
df_status.loan_status.value_counts() 

In [ ]:
objects = ('Good Debt','Bad Debt')  #change names for title, ylab!
y_pos = np.arange(len(objects))
performance = df_status.loan_status.value_counts() 
 
plt.bar(y_pos, performance, align='center', alpha=0.5)
plt.xticks(y_pos, objects)
plt.title('Loan Status')
 
plt.show()

In [ ]:
#3-2. Term
term_replace = {"term" : { ' 36 months':0,' 60 months':1 }}
df_term =df_status.replace(term_replace)
df_term.term.value_counts() 

objects = term_replace['term']
y_pos = np.arange(len(objects))
performance = df_term.term.value_counts() 
 
plt.bar(y_pos, performance, align='center', alpha=0.5)
plt.xticks(y_pos, objects)
plt.title('term')
 
plt.show()

In [ ]:
#3-3. Grade & Sub_grade
grade_replace = {"grade" : {'A':0,'B':1,'C':2,'D':3,'E':4,'F':5,'G':6 }}
df_grade =df_term.replace(grade_replace)
df_grade.grade.value_counts() 

sub_grade_replace = {"sub_grade" :{'A1':0, 'A2':1, 'A3':2, 'A4':3, 'A5':4,
                                   'B1':5, 'B2':6, 'B3':7, 'B4':8, 'B5':9,
                                   'C1':10,'C2':11,'C3':12,'C4':13,'C5':14,
                                   'D1':15,'D2':16,'D3':17,'D4':18,'D5':19,
                                   'E1':20,'E2':21,'E3':22,'E4':23,'E5':24,
                                   'F1':25,'F2':26,'F3':27,'F4':28,'F5':29,
                                   'G1':30,'G2':31,'G3':32,'G4':33,'G5':34
                                  }}
df_sub_grade =df_grade.replace(sub_grade_replace)
labels = 'A','B','C','D','E','F','G'
sizes = df_sub_grade.grade.value_counts()
colors = ['b','g','r','c','m','y','w']
plt.pie(sizes,colors=colors,labels=labels)
plt.axis('equal')
plt.tight_layout()
plt.show()

In [ ]:
objects = "B","C","A","D","E","F","G"
y_pos = np.arange(len(objects))
performance = df_sub_grade.grade.value_counts() 
 
plt.bar(y_pos, performance, align='center', alpha=0.5)
plt.xticks(y_pos, objects)
plt.title('grade')
 
plt.show()

In [ ]:
df_sub_grade.grade.value_counts() 

In [ ]:
#3-3. emp_length
emp_length_replace = { "emp_length": {"10+ years": 10,"2 years": 2, "< 1 year": 0,
                                      "3 years": 3, "1 year": 1,"5 years": 5,
                                      "4 years": 4, "7 years": 7,"8 years": 8,
                                       "6 years": 6, "9 years": 9,
                                      }}
df_emp_length =df_sub_grade.replace(emp_length_replace)
df_emp_length.emp_length.value_counts()

objects = emp_length_replace['emp_length']
y_pos = np.arange(len(objects))
performance = df_emp_length.emp_length.value_counts() 

plt.bar(y_pos, performance, align='center', alpha=0.5)
plt.xticks(y_pos, objects,rotation='vertical')
plt.title('emp_length')
 
plt.show()

In [ ]:
#3-4. home ownership
home_ownership_replace = { "home_ownership": {'MORTGAGE':2,'RENT':1,'OWN':0,'OTHER':3,'NONE':3,'ANY':3}}
df_home_ownership =df_emp_length.replace(home_ownership_replace)

objects = "MORTGAGE","RENT","OWN","OTHER"
y_pos = np.arange(len(objects))
performance = df_home_ownership.home_ownership.value_counts() 
 
plt.bar(y_pos, performance, align='center', alpha=0.5)
plt.xticks(y_pos, objects)
plt.title('home ownership')
 
plt.show()

In [ ]:
#3-4. verification_status
verification_status_replace = { "verification_status": {'Not Verified':0,'Source Verified':1,'Verified':1}}
df_verification_status =df_home_ownership.replace(verification_status_replace)

objects = "Verified","Not Verified"
y_pos = np.arange(len(objects))
performance = df_verification_status.verification_status.value_counts() 
 
plt.bar(y_pos, performance, align='center', alpha=0.5)
plt.xticks(y_pos, objects)
plt.title('Income Source Verified Status')
 
plt.show()

In [ ]:
#3-5. issue_d
df_issue_d = df_verification_status.drop(['issue_d'], axis=1)  #drop the issue_d

In [ ]:
#3-6. pymnt_plan
df_pymnt_plan = df_issue_d.drop(['pymnt_plan'], axis=1)  #drop the pymnt_plan

In [ ]:
#3-7. purpose 
le = preprocessing.LabelEncoder()
le.fit(df_pymnt_plan['purpose'])
df_pymnt_plan['purpose']=le.transform(df_pymnt_plan['purpose'])

In [ ]:
#3-8.title
df_title = df_pymnt_plan.drop(['title'], axis=1)

In [ ]:
#3-8.zip code
df_zip_code = df_title.drop(['zip_code'], axis=1)

In [ ]:
#3-9. addr_state 
le = preprocessing.LabelEncoder()
le.fit(df_zip_code['addr_state'])
df_zip_code['addr_state']=le.transform(df_zip_code['addr_state'])

In [ ]:
#3-10. earliest_cr_line
df_zip_code['earliest_cr_line']=df_zip_code['earliest_cr_line'].str.extract(r'(\d+)')
df_zip_code.earliest_cr_line.value_counts()
df_zip_code.loc[df_zip_code.earliest_cr_line.astype('int') < 1977, 'earliest_cr_line'] = 'earlier than 1977' 
le = preprocessing.LabelEncoder()
le.fit(df_zip_code['earliest_cr_line'])
df_zip_code['earliest_cr_line']=le.transform(df_zip_code['earliest_cr_line'])

In [ ]:
#3-10. initial_list_status
le = preprocessing.LabelEncoder()
le.fit(df_zip_code['initial_list_status'])
df_zip_code['initial_list_status']=le.transform(df_zip_code['initial_list_status'])

In [ ]:
#3-11. last_pymnt_d                  
df_last_pymnt_d = df_zip_code.drop(['last_pymnt_d'], axis=1)

In [ ]:
#3-12. last_credit_pull_d                            
df_last_credit_pull_d = df_last_pymnt_d.drop(['last_credit_pull_d'], axis=1)

In [ ]:
#3-13. application_type
le = preprocessing.LabelEncoder()
le.fit(df_last_credit_pull_d['application_type'])
df_last_credit_pull_d['application_type']=le.transform(df_last_credit_pull_d['application_type'])

In [ ]:
df_last_credit_pull_d.to_csv('Preprocessed_Pre_VIF.csv')

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor    

def calculate_vif_(X, thresh=5.0):
    variables = list(range(X.shape[1]))
    dropped = True
    while dropped:
        dropped = False
        vif = [variance_inflation_factor(X.iloc[:, variables].values, ix)
               for ix in range(X.iloc[:, variables].shape[1])]

        maxloc = vif.index(max(vif))
        if max(vif) > thresh:
            print('dropping \'' + X.iloc[:, variables].columns[maxloc] +
                  '\' at index: ' + str(maxloc))
            del variables[maxloc]
            dropped = True
    print(vif)
    print('Remaining variables:')
    print(X.columns[variables])
    return X.iloc[:, variables]

In [ ]:
df_Pre_Vif = df_last_credit_pull_d.loc[:, df_last_credit_pull_d.columns != 'loan_status']
calculate_vif_(df_Pre_Vif,5.0)

In [ ]:
Preprocessed_data_VIF = df_last_credit_pull_d.drop(['total_pymnt','out_prncp_inv','funded_amnt',
                                         'funded_amnt_inv','total_pymnt_inv','loan_amnt',
                                         'sub_grade','int_rate','installment','open_acc',
                                         'total_rec_prncp','earliest_cr_line','dti','revol_util',
                                         'total_acc'], axis=1) 
Preprocessed_data_VIF.to_csv('Preprocessed_Pre_GLM.csv')

In [ ]:
Preprocessed_data_Heatmap = pd.read_csv("C:/Users/Hangyu/Desktop/R_Output.csv")     #change to your own path
col_filter = list(Preprocessed_data_Heatmap.columns.values)[1:]
colormap = plt.cm.inferno
plt.figure(figsize=(12,12))
plt.title('Pearson Correlation of Features', y=1.05, size=15)
sns.heatmap(Preprocessed_data_Heatmap[col_filter].corr(),linewidths=0.1,vmax=1.0, square=True, cmap=colormap, linecolor='white', annot=True)

In [ ]:
Final_Data = Preprocessed_data_Heatmap.drop(['mths_since_last_delinq','mths_since_last_record'], axis=1) 
col_filter = list(Final_Data.columns.values)[1:]
colormap = plt.cm.inferno
plt.figure(figsize=(12,12))
plt.title('Pearson Correlation of Features', y=1.05, size=15)
sns.heatmap(Final_Data[col_filter].corr(),linewidths=0.1,vmax=1.0, square=True, cmap=colormap, linecolor='white', annot=True)
Final_Data.to_csv('Final_Data.csv')